In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
%load_ext tensorboard

# %pip install kaggle

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import pickle

2023-05-24 18:59:38.756478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 18:59:44.858928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-24 18:59:44.860195: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

**Load Data in from Kaggle**

In [3]:
# Download dataset from kaggle. Requires json credentials from kaggle.
with open('kaggle.json') as f:
    kaggle_json = json.load(f) 
os.environ['KAGGLE_USERNAME'] = kaggle_json['username']
os.environ['KAGGLE_KEY'] = kaggle_json['key']
!kaggle datasets download iamsouravbanerjee/indian-food-images-dataset

In [4]:
# Unzip data
from zipfile import ZipFile 
file_name = "indian-food-images-dataset.zip"
with ZipFile(file_name, 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 

**Set-up Training and Validation Data**

In [3]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, # Scaling
                                   shear_range=0.2, # Data Augmentation
                                   zoom_range=0.2,
                                   validation_split=0.2 # Validation set
                                  )

In [4]:
training_data_path = "Indian Food Images (All)/Indian Food Images"

target_size = (224,224)
batch_size = 32

train_set = datagen.flow_from_directory(
    training_data_path,
    target_size=target_size,
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset="training",
    seed = 24)
validation_set = datagen.flow_from_directory(
    training_data_path,
    target_size=target_size,
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset="validation",
    seed = 24)

Found 9602 images belonging to 80 classes.
Found 2387 images belonging to 80 classes.


In [5]:
image_shape = train_set.image_shape
print("Image Shape:", image_shape)

num_classes = train_set.num_classes
print("Number of Classes:", num_classes)

Image Shape: (224, 224, 3)
Number of Classes: 80


### Transfer Learning

In [6]:
from tensorflow.keras.applications import ResNet101, InceptionResNetV2, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [7]:
def build_classifier_on_top(base_model, dropout_rate=0.3, num_classes = num_classes):
    inputs = base_model.input
    x = Dense(512, activation='relu')(base_model.output)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model          

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard_resnet101 = TensorBoard(log_dir='logs/ResNet101/')
tensorboard_inception = TensorBoard(log_dir='logs/InceptionResNetV2/')
tensorboard_mobilenet = TensorBoard(log_dir='logs/MobileNetV2/')

**ResNet101**

In [13]:
resnet = ResNet101(weights='imagenet', include_top=False, input_shape=image_shape, pooling='max')
resnet.trainable = False
resnet_top = build_classifier_on_top(resnet)
resnet_top.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_top.summary()

In [16]:
history_resnet101 = resnet_top.fit(train_set, validation_data=validation_set, epochs=50, batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_resnet101])
resnet_top.save("FoodResNet101_v1.h5")
with open('train_history/history_resnet101', 'wb') as file:
    pickle.dump(history_resnet101.history, file)

Epoch 1/50


2023-05-24 15:46:18.615584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-05-24 15:46:24.493716: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55ffaec2d660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-24 15:46:24.493756: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-05-24 15:46:24.547064: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-24 15:46:25.258733: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


301/301 [==============================] - 556s 2s/step - loss: 4.4315 - accuracy: 0.0102 - val_loss: 4.3736 - val_accuracy: 0.0201
Epoch 2/50
301/301 [==============================] - 201s 667ms/step - loss: 4.3701 - accuracy: 0.0140 - val_loss: 4.3735 - val_accuracy: 0.0168
Epoch 3/50
301/301 [==============================] - 203s 676ms/step - loss: 4.3530 - accuracy: 0.0180 - val_loss: 4.3146 - val_accuracy: 0.0235
Epoch 4/50
301/301 [==============================] - 204s 678ms/step - loss: 4.3346 - accuracy: 0.0204 - val_loss: 4.3076 - val_accuracy: 0.0239
Epoch 5/50
301/301 [==============================] - 203s 674ms/step - loss: 4.3218 - accuracy: 0.0177 - val_loss: 4.3208 - val_accuracy: 0.0205
Epoch 6/50
301/301 [==============================] - 202s 672ms/step - loss: 4.3113 - accuracy: 0.0205 - val_loss: 4.2938 - val_accuracy: 0.0264
Epoch 7/50
301/301 [==============================] - 202s 672ms/step - loss: 4.3006 - accuracy: 0.0234 - val_loss: 4.2702 - val_accuracy:

NameError: name 'mobilenet_top' is not defined

**InceptionResNetV2**

In [12]:
inception_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=image_shape, pooling='max')
inception_resnet.trainable = False
inception_resnet_top = build_classifier_on_top(inception_resnet)
inception_resnet_top.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# inception_resnet_top.summary()

In [13]:
history_inception_resnet = inception_resnet_top.fit(train_set, validation_data=validation_set, epochs=50, batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_inception])
inception_resnet_top.save("InceptionResNetV2_v1.h5")
with open('train_history/history_inception_resnet', 'wb') as file:
    pickle.dump(history_inception_resnet.history, file)

Epoch 1/50
301/301 [==============================] - 578s 2s/step - loss: 4.4162 - accuracy: 0.0245 - val_loss: 4.1882 - val_accuracy: 0.0524
Epoch 2/50
301/301 [==============================] - 220s 731ms/step - loss: 4.0315 - accuracy: 0.0516 - val_loss: 3.6274 - val_accuracy: 0.1089
Epoch 3/50
301/301 [==============================] - 218s 726ms/step - loss: 3.7871 - accuracy: 0.0778 - val_loss: 3.4747 - val_accuracy: 0.1441
Epoch 4/50
301/301 [==============================] - 217s 721ms/step - loss: 3.6732 - accuracy: 0.0899 - val_loss: 3.3349 - val_accuracy: 0.1684
Epoch 5/50
301/301 [==============================] - 218s 723ms/step - loss: 3.6117 - accuracy: 0.1024 - val_loss: 3.2117 - val_accuracy: 0.1848
Epoch 6/50
301/301 [==============================] - 217s 722ms/step - loss: 3.5151 - accuracy: 0.1157 - val_loss: 3.2137 - val_accuracy: 0.1910
Epoch 7/50
301/301 [==============================] - 217s 721ms/step - loss: 3.4762 - accuracy: 0.1227 - val_loss: 3.0882 - va

**MobileNetV2**

In [ ]:
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=image_shape, pooling='max')
mobilenet.trainable = False
mobilenet_top = build_classifier_on_top(mobilenet)
mobilenet_top.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# mobilenet_top.summary()

In [ ]:
history_mobilenet = mobilenet_top.fit(train_set, validation_data=validation_set, epochs=50, batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_mobilenet])
mobilenet_top.save("MobileNetV2_v1.h5")
with open('train_history/history_mobilenet', 'wb') as file:
    pickle.dump(history_mobilenet.history, file)

Epoch 1/50
301/301 [==============================] - 215s 703ms/step - loss: 4.2494 - accuracy: 0.0392 - val_loss: 3.4993 - val_accuracy: 0.1362
Epoch 2/50
301/301 [==============================] - 209s 695ms/step - loss: 3.5477 - accuracy: 0.1102 - val_loss: 3.0887 - val_accuracy: 0.1956
Epoch 3/50
301/301 [==============================] - 209s 693ms/step - loss: 3.1934 - accuracy: 0.1794 - val_loss: 2.7699 - val_accuracy: 0.2614
Epoch 4/50
301/301 [==============================] - 207s 689ms/step - loss: 2.9958 - accuracy: 0.2134 - val_loss: 2.6422 - val_accuracy: 0.2811
Epoch 5/50
301/301 [==============================] - 208s 690ms/step - loss: 2.8932 - accuracy: 0.2419 - val_loss: 2.5431 - val_accuracy: 0.2970
Epoch 6/50
301/301 [==============================] - 207s 689ms/step - loss: 2.7767 - accuracy: 0.2650 - val_loss: 2.4989 - val_accuracy: 0.3364
Epoch 7/50
301/301 [==============================] - 209s 694ms/step - loss: 2.6982 - accuracy: 0.2798 - val_loss: 2.4301 -

### Fine-tuning

In [18]:
mobilenet_top = keras.models.load_model("MobileNetV2_v1.h5")

In [19]:
mobilenet_top.trainable = True
mobilenet_top.compile(optimizer=keras.optimizers.Adam(1e-4),  # Very low learning rate
               loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# mobilenet_top.summary() # check layers are unfreezed

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=7)
tensorboard_mobilenet_ft = TensorBoard(log_dir='logs/mobilenet_ft/')

history_mobilenet_ft = mobilenet_top.fit(train_set,
                                         validation_data=validation_set,
                                         epochs=100,
                                         batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_mobilenet_ft])
mobilenet_top.save("MobileNetV2_ft.h5")
with open('train_history/history_mobilenet_ft', 'wb') as file:
    pickle.dump(history_mobilenet_ft.history, file)

Epoch 1/100
301/301 [==============================] - 240s 712ms/step - loss: 2.7908 - accuracy: 0.3090 - val_loss: 2.4025 - val_accuracy: 0.3695
Epoch 2/100
301/301 [==============================] - 210s 698ms/step - loss: 2.3279 - accuracy: 0.3796 - val_loss: 2.4631 - val_accuracy: 0.3532
Epoch 3/100
301/301 [==============================] - 211s 699ms/step - loss: 2.1745 - accuracy: 0.4152 - val_loss: 2.1301 - val_accuracy: 0.4344
Epoch 4/100
301/301 [==============================] - 210s 696ms/step - loss: 2.0826 - accuracy: 0.4363 - val_loss: 2.1066 - val_accuracy: 0.4437
Epoch 5/100
301/301 [==============================] - 209s 694ms/step - loss: 1.9559 - accuracy: 0.4653 - val_loss: 2.0153 - val_accuracy: 0.4667
Epoch 6/100
301/301 [==============================] - 212s 705ms/step - loss: 1.8806 - accuracy: 0.4817 - val_loss: 1.9108 - val_accuracy: 0.4881
Epoch 7/100
301/301 [==============================] - 208s 692ms/step - loss: 1.8259 - accuracy: 0.4932 - val_loss: 1

**Custom CNN**

In [27]:
# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# Flatten the feature maps
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(80, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_422 (Conv2D)         (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_423 (Conv2D)         (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_424 (Conv2D)         (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 26, 26, 128)     

In [28]:
early_stopping = EarlyStopping(monitor='val_loss', patience=7)
tensorboard_mycnn = TensorBoard(log_dir='logs/my_cnn/')
my_cnn_history = model.fit(train_set,
                           validation_data=validation_set,
                           epochs=100,
                           batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_mycnn])
model.save("IndianFoodNet_v1.h5")
with open('train_history/my_cnn', 'wb') as file:
    pickle.dump(my_cnn_history.history, file)

Epoch 1/100
301/301 [==============================] - 193s 631ms/step - loss: 4.3178 - accuracy: 0.0174 - val_loss: 4.1760 - val_accuracy: 0.0226
Epoch 2/100
301/301 [==============================] - 189s 628ms/step - loss: 4.1609 - accuracy: 0.0265 - val_loss: 4.0800 - val_accuracy: 0.0314
Epoch 3/100
301/301 [==============================] - 190s 632ms/step - loss: 4.0998 - accuracy: 0.0342 - val_loss: 4.0400 - val_accuracy: 0.0415
Epoch 4/100
301/301 [==============================] - 190s 633ms/step - loss: 4.0266 - accuracy: 0.0479 - val_loss: 4.0110 - val_accuracy: 0.0452
Epoch 5/100
301/301 [==============================] - 191s 636ms/step - loss: 3.9204 - accuracy: 0.0586 - val_loss: 3.8454 - val_accuracy: 0.0683
Epoch 6/100
301/301 [==============================] - 189s 630ms/step - loss: 3.8085 - accuracy: 0.0739 - val_loss: 3.6920 - val_accuracy: 0.0926
Epoch 7/100
301/301 [==============================] - 190s 630ms/step - loss: 3.6929 - accuracy: 0.0939 - val_loss: 3

In [39]:
def eval_model(model):
    print("Training Accuracy:")
    print(model.evaluate(train_set))
    print("Validation Accuracy:")
    print(model.evaluate(validation_set))

In [48]:
eval_model(model)

Training Accuracy:
301/301 [==============================] - 163s 540ms/step - loss: 1.1105 - accuracy: 0.7320
[1.110529899597168, 0.7320349812507629]
Validation Accuracy:
75/75 [==============================] - 28s 370ms/step - loss: 3.4585 - accuracy: 0.2698
[3.458545684814453, 0.2697947323322296]


In [43]:
eval_model(keras.models.load_model("FoodResNet101_v1.h5"))

Training Accuracy:
301/301 [==============================] - 170s 556ms/step - loss: 4.1404 - accuracy: 0.0505
[4.140432357788086, 0.050510309636592865]
Validation Accuracy:
75/75 [==============================] - 30s 402ms/step - loss: 4.1839 - accuracy: 0.0364
[4.1839470863342285, 0.036447424441576004]


In [45]:
eval_model(keras.models.load_model("InceptionResNetV2_v1.h5"))

Training Accuracy:
301/301 [==============================] - 172s 560ms/step - loss: 2.6784 - accuracy: 0.3091
[2.6784048080444336, 0.30910226702690125]
Validation Accuracy:
75/75 [==============================] - 30s 395ms/step - loss: 2.7630 - accuracy: 0.2866
[2.7629549503326416, 0.28655216097831726]


In [46]:
eval_model(keras.models.load_model("MobileNetV2_v1.h5"))

Training Accuracy:
301/301 [==============================] - 163s 539ms/step - loss: 1.4870 - accuracy: 0.5891
[1.4870275259017944, 0.5891481041908264]
Validation Accuracy:
75/75 [==============================] - 28s 376ms/step - loss: 2.0938 - accuracy: 0.4378
[2.0937535762786865, 0.4377880096435547]


In [47]:
eval_model(keras.models.load_model("MobileNetV2_ft.h5"))

Training Accuracy:
301/301 [==============================] - 164s 541ms/step - loss: 0.7375 - accuracy: 0.7920
[0.7375173568725586, 0.7920224666595459]
Validation Accuracy:
75/75 [==============================] - 29s 381ms/step - loss: 1.6677 - accuracy: 0.5710
[1.667717695236206, 0.571009635925293]
